<a href="https://colab.research.google.com/github/tangluna/CertificateGenerator/blob/master/CertificateGenerator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Generating Certificates from CSV File Data
By: Tarang Lunawat

This program reads in names and Python achievement levels from a CSV file, and then generates and downloads a personalized certificate PDF for each of them. The certificate includes the name, level of Python mastery, logos, and a signature, and the PDF is named using the recipient name.

There's an example certificate in the GitHub README

First, import the relavent libraries

In [ ]:
!pip install fpdf #bash command to get the fpdf library

from fpdf import FPDF
from google.colab import files
from datetime import datetime as dt
import numpy as np
from PIL import Image, ImageDraw
import pandas as pd
import pytz

  Created wheel for fpdf: filename=fpdf-1.7.2-py2.py3-none-any.whl size=40720 sha256=d7baaabf434755523e78c0d807c52a6ac6c9eca365ae8a728a184437b5cded1e
  Stored in directory: /root/.cache/pip/wheels/9a/e9/77/4554ff5c99bc3f487c8d69620d8c41d99d54e9c54ab20ef4c9
Successfully built fpdf


#Image Imports
First, we have to import all the images. The first image (‘e42aab1b15084b3afcd98a88ce07ce01.jpg’) is the background for the certificate. Feel free to change that to any other one of your preference by replacing the link and the name of the image later on in the program. You may also have to adjust some text colors to retain the cohesive look of the certificate.

The last image uploaded is an image of the signature used on the certificate. **It is currently a placeholder. Be sure to upload a real signature and replace the name in the signatureImgName variable**

The other three images are the logos included in the bottom of the certificate. The first and third are used as is, but the second one is edited a little later on.

In [ ]:
!wget "https://i.pinimg.com/originals/e4/2a/ab/e42aab1b15084b3afcd98a88ce07ce01.jpg" # certificate background

--2020-06-28 22:43:49--  https://i.pinimg.com/originals/e4/2a/ab/e42aab1b15084b3afcd98a88ce07ce01.jpg
Resolving i.pinimg.com (i.pinimg.com)... 104.18.15.176, 104.18.14.176, 2a04:4e42:2f::84
Connecting to i.pinimg.com (i.pinimg.com)|104.18.15.176|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 55062 (54K) [image/jpeg]
Saving to: ‘e42aab1b15084b3afcd98a88ce07ce01.jpg’

e42aab1b15084b3afcd 100%[===================>]  53.77K  --.-KB/s    in 0s      

2020-06-28 22:43:49 (135 MB/s) - ‘e42aab1b15084b3afcd98a88ce07ce01.jpg’ saved [55062/55062]



In [ ]:
!wget "https://www.csr.utexas.edu/internship/images/tsgc_logo.png" # tsgc logo

--2020-06-28 23:46:56--  https://www.csr.utexas.edu/internship/images/tsgc_logo.png
Resolving www.csr.utexas.edu (www.csr.utexas.edu)... 128.83.21.174
Connecting to www.csr.utexas.edu (www.csr.utexas.edu)|128.83.21.174|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26636 (26K) [image/png]
Saving to: ‘tsgc_logo.png’

tsgc_logo.png       100%[===================>]  26.01K  --.-KB/s    in 0s      

2020-06-28 23:46:56 (201 MB/s) - ‘tsgc_logo.png’ saved [26636/26636]



In [ ]:
!wget "http://www.tsgc.utexas.edu/sees-internship/graphics/sees-box-240x240.png" # sees logo

--2020-06-28 23:58:33--  http://www.tsgc.utexas.edu/sees-internship/graphics/sees-box-240x240.png
Resolving www.tsgc.utexas.edu (www.tsgc.utexas.edu)... 146.6.34.4
Connecting to www.tsgc.utexas.edu (www.tsgc.utexas.edu)|146.6.34.4|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 33914 (33K) [image/png]
Saving to: ‘sees-box-240x240.png’

sees-box-240x240.pn 100%[===================>]  33.12K  --.-KB/s    in 0.07s   

2020-06-28 23:58:34 (453 KB/s) - ‘sees-box-240x240.png’ saved [33914/33914]



In [ ]:
!wget "https://upload.wikimedia.org/wikipedia/commons/thumb/7/77/University_of_Texas_at_Austin_logo.svg/1280px-University_of_Texas_at_Austin_logo.svg.png" # ut austin logo

--2020-06-29 01:14:38--  https://upload.wikimedia.org/wikipedia/commons/thumb/7/77/University_of_Texas_at_Austin_logo.svg/1280px-University_of_Texas_at_Austin_logo.svg.png
Resolving upload.wikimedia.org (upload.wikimedia.org)... 103.102.166.240, 2620:0:861:ed1a::2:b
Connecting to upload.wikimedia.org (upload.wikimedia.org)|103.102.166.240|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 79446 (78K) [image/png]
Saving to: ‘1280px-University_of_Texas_at_Austin_logo.svg.png’

1280px-University_o 100%[===================>]  77.58K   153KB/s    in 0.5s    

2020-06-29 01:14:39 (153 KB/s) - ‘1280px-University_of_Texas_at_Austin_logo.svg.png’ saved [79446/79446]



This last upload is for an image of the signature to go on the certificates. **Make sure to change it to an actual signature - right now it is just a sample signature (the signature reads 'Signature').** If the signature is accessible through URL, just change the URL in the first line.

Otherwise, uncomment the middle section and *comment out the first two lines*. Now, running the cell will allow you to upload a file to use as the signature image.

Remember to change the variable in the last line. If it is left blank as it is originally, the text under the signature image will just read 'Signature'. If it is replaced with a name (or any other text), that text will be shown under the signature line.

In [ ]:
!wget "https://i.ya-webdesign.com/images/sample-signature-png-9.png" # sample signature - CHANGE THIS TO A REAL SIGNATURE FOR OFFICIAL CERIFICATES
signatureImgName = "sample-signature-png-9.png" # MAKE SURE TO ALSO CHANGE THE NAME IN THIS VARIABLE TO MATCH THE UPLOADED IMAGE

## UNCOMMENT THIS SECTION IF YOU WOULD PREFER TO UPLOAD A FILE - MAKE SURE TO COMMENT OUT THE FIRST TWO LINES IF SO
'''
uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))
  
signatureImgName = list(uploaded.keys())[0]
'''

nameOfSigner = '' # REPLACE THIS (otherwise, the text under the signature will just read 'Signature')

--2020-06-29 05:05:24--  https://i.ya-webdesign.com/images/sample-signature-png-9.png
Resolving i.ya-webdesign.com (i.ya-webdesign.com)... 172.67.223.91, 104.27.137.3, 104.27.136.3, ...
Connecting to i.ya-webdesign.com (i.ya-webdesign.com)|172.67.223.91|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5996 (5.9K) [image/png]
Saving to: ‘sample-signature-png-9.png.1’

sample-signature-pn 100%[===================>]   5.86K  --.-KB/s    in 0s      

2020-06-29 05:05:24 (61.5 MB/s) - ‘sample-signature-png-9.png.1’ saved [5996/5996]



#Photo Editing
The SEES logo has a non-transparent background, so I created a function to edit that photo into a circle for a nicer look on the certificate. This function takes in the image, takes off the edges, and then masks it into a circle. The masked image is saved as a new file.

The last line of this code block calls the function on the SEES logo.

In [ ]:
def make_image_circle(imageName, x_trim = 0, y_trim = 0):
  img = Image.open(imageName) # opening image for editing
  h,w = img.size
  img = img.crop((x_trim, y_trim, w - x_trim, h - y_trim)).convert("RGB") # cropping the image and converting it into RGB colors
  npImage = np.array(img) # converts image into an array
  h,w = img.size # size variables redefined after image is cropped

  alpha = Image.new('L', img.size, 0) # creates new empty image
  draw = ImageDraw.Draw(alpha)
  draw.pieslice([0,0,h,w],0,360,fill=255) # draws white circle on empty image

  npAlpha = np.array(alpha) # converts new image into an array

  npImage = np.dstack((npImage, npAlpha)) # arrays are stacked - the first image's colors are kept only where the second image had a white fill 

  Image.fromarray(npImage).save(imageName[:-4] + "_circle.png") # converts array back into an image and saves it

make_image_circle("sees-box-240x240.png", 10, 10) # actually edits the image

#Making the Certificate PDF
This function is what actually generates certificates! It takes in the name of the recipient and level of certificate, and then generates a new pdf. The pdf is saved as **'certificate_Lastname_Firstname.pdf'** and is downloaded to the user's computer.

Currently, the signature is a sample image - make sure to upload an actual signature above and change the name that is used above!

This certificate also includes a date - it is autogenerated from system time and converted to US - Central (Austin, TX) time.

The numbers used to get the right spacing are very finicky - *edit at your own risk*.

In [1]:
def make_certificate(fName, lName, level):
  pdf = FPDF('L', 'mm', 'Letter') # initializes PDF to landscape orientation, letter size, and measurements in mm
  pdf.add_page()
  pdf.image("e42aab1b15084b3afcd98a88ce07ce01.jpg", 0, 0, 279, 215) # adds background certificate image
  pdf.set_font('Times', 'B', 50) # setting font
  pdf.ln(49) # spacing
  pdf.set_text_color(0, 0, 50) # setting text color
  pdf.cell(0, 0, 'Certificate of Completion', 0, 1, 'C') # title text
  pdf.set_font('Helvetica', '', 16)
  pdf.ln(26)
  pdf.set_text_color(0, 0, 0)
  pdf.cell(0, 0, 'This certifies that', 0, 1, 'C') # certification text
  pdf.set_font('Times', 'B', 50)
  pdf.set_text_color(0, 0, 50)
  pdf.ln(15)
  pdf.cell(0, 0, fName + ' ' + lName, 0, 1, 'C') # name of recipient
  pdf.set_font('Helvetica', '', 16)
  pdf.set_text_color(0, 0, 0)
  pdf.ln(15)
  pdf.cell(0, 8, 'has completed Lesson 2: Python, ' + level + ' Level at the NASA Texas Space', 0, 1, 'C') # certificate details
  pdf.cell(0, 8, 'Grant Consortium STEM Enhancement in Earth Science Summer Intern Program.', 0, 1, 'C')
  pdf.ln(17)
  date = dt.now(pytz.timezone('US/Central')) # getting date object (automatic - uses US Central time)
  pdf.cell(90, 10, date.strftime('%B %d, %Y'), 0, 0, 'R') # formats and prints date
  pdf.line(50, 157, 100, 157)
  pdf.cell(69, 10, '', 0, 0)
  # pdf.cell(110, 10, signatureText, 0, 0, 'L')
  pdf.image(signatureImgName, 170, 137, 50) # signature image
  pdf.line(170, 157, 220, 157)
  pdf.ln(7)
  pdf.set_font_size(12)
  pdf.cell(90, 10, 'Date', 0, 0, 'R') # text under date
  if (nameOfSigner != ''): # text under signature depending on what the variable holds
    pdf.cell(70, 10, '', 0, 0)
    pdf.cell(90, 10, nameOfSigner, 0, 0, 'L')
  else:
    pdf.cell(90, 10, "Signature", 0, 0, 'R')
  pdf.image("sees-box-240x240_circle.png", 124, 140, 25) # adding logos
  pdf.image("tsgc_logo.png", 70, 160, 65)
  pdf.image("1280px-University_of_Texas_at_Austin_logo.svg.png", 150, 171, 50)
  pdf.output('certificate_' + lName + '_' + fName + '.pdf', 'F') # saving PDF
  files.download('certificate_' + lName + '_' + fName + '.pdf') # downloading PDF to computer

#CSV File Processing
This program accepts data as a CSV file. The titles of the CSV file columns should be "First Name", "Last Name", and "Level". **If your CSV file has different headings, this program will not work.**

Here's an example of a CSV file that works: https://github.com/tangluna/CertificateGenerator/blob/master/ExampleCertificateInputData.csv

Don't worry about what your CSV file is titled - the program automatically stores the name!

In [ ]:
uploaded = files.upload() # uploads files

for fn in uploaded.keys(): # prints out all the files that were uploaded
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))
  
certificateInfoFile = list(uploaded.keys())[0] # stores first (and only) file's name

Saving CertificateInfo.csv to CertificateInfo.csv
User uploaded file "CertificateInfo.csv" with length 95 bytes


I used the pandas library to parse the CSV file. It returns a DataFrame, which is stored in data.

In [ ]:
data = pd.read_csv(certificateInfoFile) # parses and stores CSV data

Finally, where the magic happens! This loops through all the rows in the CSV file, and subsequently in the DataFrame. For each row, it generates a certificate by calling the make_certificate function with arguments from what is stored in that row of the CSV file.

In [ ]:
for ind in data.index: # loops through the CSV rows (excluding the first)
  make_certificate(data['First Name'][ind], data['Last Name'][ind], data['Level'][ind]) # calls function to generate and download certificate with data from that row


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>